# BENEFÍCIOS AO CIDADÃO
A seguinte tarefa consiste em:

1.0. planejar a tarefa de extração, tratamento e disponibilização dos dados de benefícios (calendário de entregas);

1.1. obter os dados mais recentes dos benefícios sociais disponíveis na página: http://www.transparencia.gov.br/download-de-dados em BENEFÍCIOS AO CIDADÃO;

1.2. tratar os dados obtidos para obter apenas o número de identificação dos beneficiários e os períodos em que os benefícios foram recebidos;

1.3. disponibilizar em um só arquivo (txt ou csv) tabela com beneficiários, benefícios e períodos de recebimento;

1.4. documentar processos de trabalho (obtenção, tratamento, disponibilização);

1.5. avaliar viabilidade da automação dos processos (com o supervisor);

2.1. automatizar a obtenção dos dados;

2.2. automatizar o tratamento de dados;

2.3. automatizar a disponibilização de dados;

2.4. documentar processos de trabalho (automatização da obtenção, tratamento, disponibilização);

2.4. avaliar atualização do processo (com o supervisor);

3.1 avaliar projeto (com o supervisor);

3.2. inclusão do projeto no processo de trabalho LAB;


* Colunas para utilizar: "NOME BENEFICIÁRIO", "CPF BENEFICIÁRIO", "NIS BENEFICIÁRIO".
* Criar coluna "BENEFÍCIO", ao qual no arquivo final contenham colunas referentes a cada benefício dos dados coletados e o valor de "0" ou "1", caso o beneficiário tenha ou não o benefício respectivo a coluna.

In [1]:
# Abrindo conexão com o Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Importação de Bibliotecas
import glob # O glob é uma biblioteca para coletar e manipular o endereço de pastas no computador
import os
import pandas as pd
import numpy as np

In [28]:
# Variáveis globais.
caminho_padrao = "/content/drive/MyDrive/Projetos_Tassio_Sales/beneficios_ao_cidadao-12-2021/dados_brutos/"
caminho_salvar = "/content/drive/MyDrive/Projetos_Tassio_Sales/beneficios_ao_cidadao-12-2021/dados_tratamento_primario/"

colunas_1 = ["NOME BENEFICIÁRIO", "CPF BENEFICIÁRIO", "NIS BENEFICIÁRIO"] #aux, bpc
colunas_2 = ["NOME FAVORECIDO", "CPF FAVORECIDO", "NIS FAVORECIDO"] #bfpag, bfsaq, sgdf
colunas_3 = ["NOME FAVORECIDO", "NIS FAVORECIDO"] #gs, peti

colunas_renomear = ["NOME_BENEFICIARIO", "CPF_BENEFICIARIO", "NIS_BENEFICIARIO", "BENEFICIO"]

In [5]:
# Função para o menu de escolha do benefício para coleta.
def menuBeneficio():
  # Menu para escolha do benefício.
  print("1 - Auxílio emergencial")
  print("2 - Bolsa Família - Pagamentos")
  print("3 - Bolsa Família - Saques")
  print("4 - Garantia Safra")
  print("5 - Seguro Defeso (Pescador Artesanal)")
  print("6 - Erradicação do Trabalho Infantil (PETI)")
  print("7 - Benefício de Prestação Continuada (BPC)")

  while True:
    try:
      opt = int(input("Digite a opção desejada: "))
      if opt == 1:
        print("1 - Auxílio emergencial")
        beneficio = "auxilio-emergencial"
        nome_beneficio = "Auxilio_Emergencial"
        colunas = colunas_1
        break
      elif opt == 2:
        print("2 - Bolsa Família - Pagamentos")
        beneficio = "bolsa-familia-pagamentos"
        nome_beneficio = "Bolsa_Familia_Pagamentos"
        colunas = colunas_2
        break
      elif opt == 3:
        print("3 - Bolsa Família - Saques")
        beneficio = "bolsa-familia-saques"
        nome_beneficio = "Bolsa_Familia_Saques"
        colunas = colunas_2
        break
      elif opt == 4:
        print("4 - Garantia Safra")
        beneficio = "garantia-safra"
        nome_beneficio = "Garantia_Safra"
        colunas = colunas_3
        break
      elif opt == 5:
        print("5 - Seguro Defeso (Pescador Artesanal)")
        beneficio = "seguro-defeso"
        nome_beneficio = "Seguro_Defeso_(Pescador_Artesanal)"
        colunas = colunas_2
        break
      elif opt == 6:
        print("6 - Erradicação do Trabalho Infantil (PETI)")
        beneficio = "peti"
        nome_beneficio = "Erradicacao_do_Trabalho_Infantil_(PETI)"
        colunas = colunas_3
        break
      elif opt == 7:
        print("7 - Benefício de Prestação Continuada (BPC)")
        beneficio = "bpc"
        nome_beneficio = "Beneficio_de_Prestacao_Continuada_(BPC)"
        colunas = colunas_1
        break
    except ValueError as V:
      print(V)
      print(V)
      print("Valor digitado incorreto apenas numeros inteiros:")
      pass
    except Exception as E:
      print(E)
      print(E)
  return beneficio, nome_beneficio, colunas

def listarArquivos(path):
  # Listando arquivos contidos na pasta.
  caminho = path + "/*.csv"
  arquivos = glob.glob(caminho)
  print("Arquivos Contidos: {}".format(arquivos))
  return arquivos

def lerArquivo(file, colunas):
    # Realiza a leitura do arquivo.
    dados = pd.read_csv(file, sep=";", encoding="latin-1", dtype=str, usecols=colunas)
    return dados

def concatenarArquivos(data):
  # Concatena arquivos em um único dataframe.
  df = pd.concat(data, ignore_index=True)
  return df

def criaColunasFaltantes(df_1, colunas):
  if list(df_1.columns) == colunas and len(df_1.columns) < 3:
    df_1["CPF FAVORECIDO"] = "XXX.XXX.XXX-XX"
    print("Adicionando colunas CPF")
    print(f"As colunas são: {df_1.columns}")
  else:
    print("Todas as colunas OK")
    print(f"As colunas são: {df_1.columns}")
  
  return df_1

def criaColunaBeneficio(df_2, nome_beneficio):
  print(f"Criou coluna com o benefício: {nome_beneficio}")
  df_2["BENEFICIO"] = nome_beneficio
  return df_2

def salvarArquivo(dataset, file_name):
  # Salva o arquivo único com o nome e na pasta do respectivo benefício.
  dataset.to_csv(file_name, sep=";", encoding='latin-1', index=False)
  
  print() # Printa vazio, ou seja, pula uma linha na visualização.

  # Informar dados gerais sobre o arquivo gerado.
  return dataset.info() # memory_usage = informa a quantidade de memória usada.

In [30]:
# Seleciona o benefício e realiza o tratamento inicial dos dados.
beneficio, nome_beneficio, colunas = menuBeneficio()

# Definição da pasta para ser acessada na verificação dos arquivos contidos.
pasta = caminho_padrao + beneficio

print("\n##########--> Benefício: {} <--##########".format(beneficio))
if beneficio == "peti" or beneficio == "garantia-safra" or beneficio == "bolsa-familia-pagamentos" or beneficio == "bolsa-familia-saques" or beneficio == "seguro-defeso":
    reordenar_como = colunas_2
elif beneficio == "auxilio-emergencial" or beneficio == "bpc":
    reordenar_como = colunas_1

# Geração pasta para ser acessada.
print("Acessou a pasta: {}\n".format(pasta))

arquivos = listarArquivos(pasta)
print()

for arquivo in arquivos:
    name = arquivo.split('/')[-1].split('.')[0]
    print(f"### Lendo arquivo: {name}")
    dados = lerArquivo(arquivo, colunas)
    
    dados.drop_duplicates(inplace=True)
    dados = dados[colunas] # Ordena colunas para verificação e criação de colunas faltantes.
    
    df = criaColunasFaltantes(dados, colunas)
    df = df[reordenar_como] # Ordena colunas
    
    df_1 = criaColunaBeneficio(df, nome_beneficio)
    df_1.columns = colunas_renomear # Renomeia as colunas do arquivo, mantendo um padrão para todos.
    
    df_1["CPF_BENEFICIARIO"].fillna(value="XXX.XXX.XXX-XX", inplace=True)
    df_1["NIS_BENEFICIARIO"].fillna(value='XXXXXXXXXXX', inplace=True)
    df_1["CPF_BENEFICIARIO"].replace(r"[*]", "X", inplace=True, regex=True)
    
    # Salva o arquivo único na pasta do respectivo benefício.
    file_name = caminho_salvar + beneficio + "/" + name + ".txt"
    print("### Salvando arquivo: {}".format(file_name))
    df_2 = salvarArquivo(df_1, file_name)
    
    print() # Printa vazio, ou seja, pula uma linha na visualização.

1 - Auxílio emergencial
2 - Bolsa Família - Pagamentos
3 - Bolsa Família - Saques
4 - Garantia Safra
5 - Seguro Defeso (Pescador Artesanal)
6 - Erradicação do Trabalho Infantil (PETI)
7 - Benefício de Prestação Continuada (BPC)
Digite a opção desejada: 2
2 - Bolsa Família - Pagamentos

##########--> Benefício: bolsa-familia-pagamentos <--##########
Acessou a pasta: /content/drive/MyDrive/Projetos_Tassio_Sales/beneficios_ao_cidadao-12-2021/dados_brutos/bolsa-familia-pagamentos

Arquivos Contidos: ['/content/drive/MyDrive/Projetos_Tassio_Sales/beneficios_ao_cidadao-12-2021/dados_brutos/bolsa-familia-pagamentos/202103_BolsaFamilia_Pagamentos.csv', '/content/drive/MyDrive/Projetos_Tassio_Sales/beneficios_ao_cidadao-12-2021/dados_brutos/bolsa-familia-pagamentos/202104_BolsaFamilia_Pagamentos.csv', '/content/drive/MyDrive/Projetos_Tassio_Sales/beneficios_ao_cidadao-12-2021/dados_brutos/bolsa-familia-pagamentos/202101_BolsaFamilia_Pagamentos.csv', '/content/drive/MyDrive/Projetos_Tassio_Sales